In [1]:
from preamble import *

In [2]:
#------------------------------------------------------------#
MaxIntFilter = 64 #knots
#------------------------------------------------------------#

In [3]:
# Load the ibtracs dataset
dataset = pd.read_csv('Data/IBTrACS/data_ibtracs_JTWC.csv')

# Apply VMAX corrections
dataset['A_VMAX'] = 0.1884 * dataset['VMAX']**1.288
dataset['VMAX2'] = 0
dataset.loc[dataset['YEAR'] < 1968, 'VMAX2'] = 0.4 * dataset['VMAX'] + 0.6 * dataset['A_VMAX']
dataset.loc[dataset['YEAR'].between(1968,1972), 'VMAX2'] = 0.8 * dataset['VMAX'] + 0.2 * dataset['A_VMAX']
dataset.loc[dataset['YEAR'] > 1972, 'VMAX2'] = dataset['VMAX']
dataset = dataset.drop(['VMAX','A_VMAX'], axis=1)
dataset = dataset.rename(columns={'VMAX2': 'VMAX'})

In [4]:
# Determine maximum intensity of the each cyclone
max_int = dataset.groupby('CycNo')['VMAX'].max().rename('VMAX').reset_index()

# Remove cyclones with maximum intensity lower than defined
max_int = max_int[max_int['VMAX'] > MaxIntFilter]
dataset = dataset[dataset['CycNo'].isin(max_int['CycNo'])]

In [5]:
dataset['CycNo'].nunique()

938

In [6]:
dataset.columns

Index(['BASIN', 'CycNo', 'DIST2LAND', 'GENESIS', 'LANDFALL', 'LAT', 'LON',
       'S_TIME', 'YEAR', 'VMAX'],
      dtype='object')

In [34]:
def influence(data):
    peak_time = data['S_TIME'][data['VMAX'].idxmax()]
    try:
        landfall_time = data[data['DIST2LAND'] == 0].iloc[0]['S_TIME']
        if landfall_time <= peak_time:
            return True
        else:
            return False
    except:
        return False
    

In [35]:
effect_data = dataset.groupby('CycNo').apply(influence)

In [38]:
effect_data.reset_index()

,CycNo,0
0,5453.0,False
1,5459.0,False
2,5468.0,False
3,5481.0,False
4,5490.0,False
5,5495.0,True
6,5504.0,False
7,5506.0,True
8,5511.0,False
9,5517.0,False


In [36]:
len(effect_data[effect_data == True])

110

In [18]:
test_set = dataset[dataset['CycNo']==10890]

In [19]:
test_set

,BASIN,CycNo,DIST2LAND,GENESIS,LANDFALL,LAT,LON,S_TIME,YEAR,VMAX
40193,2.0,10890.0,511.0,2.0,476.0,16.900000,114.999997,50679.50,1997.0,25.000000
40194,2.0,10890.0,476.0,2.0,393.0,17.500000,115.499997,50679.75,1997.0,25.000000
40195,2.0,10890.0,389.0,2.0,319.0,18.100000,116.399997,50680.00,1997.0,30.000000
40196,2.0,10890.0,322.0,2.0,322.0,18.700000,117.299997,50680.25,1997.0,30.000000
40197,2.0,10890.0,353.0,2.0,353.0,19.100000,117.199997,50680.50,1997.0,35.000001
40198,2.0,10890.0,400.0,2.0,334.0,19.600000,116.699997,50680.75,1997.0,35.000001
40199,2.0,10890.0,334.0,2.0,266.0,20.000000,115.499997,50681.00,1997.0,40.000001
40200,2.0,10890.0,266.0,2.0,214.0,20.100000,114.299997,50681.25,1997.0,45.000001
40201,2.0,10890.0,214.0,2.0,156.0,20.100000,113.199997,50681.50,1997.0,50.000001
40202,2.0,10890.0,160.0,2.0,68.0,20.100000,112.299997,50681.75,1997.0,55.000001


In [20]:
test_set['S_TIME'][test_set['VMAX'].idxmax()]

50682.25

In [26]:
test_set[test_set['DIST2LAND'] == 0].iloc[0]['S_TIME']

50682.25

In [15]:
np.unique(dataset[dataset['LANDFALL']==0]['CycNo'])

array([  5459.,   5468.,   5481.,   5490.,   5495.,   5504.,   5506.,
         5511.,   5521.,   5527.,   5563.,   5568.,   5575.,   5583.,
         5609.,   5617.,   5624.,   5636.,   5649.,   5664.,   5668.,
         5675.,   5679.,   5684.,   5693.,   5697.,   5712.,   5715.,
         5743.,   5767.,   5769.,   5774.,   5780.,   5784.,   5800.,
         5810.,   5813.,   5818.,   5852.,   5866.,   5892.,   5893.,
         5897.,   5908.,   5915.,   5959.,   5984.,   5994.,   6000.,
         6002.,   6003.,   6013.,   6014.,   6015.,   6021.,   6028.,
         6038.,   6070.,   6074.,   6078.,   6084.,   6090.,   6094.,
         6102.,   6118.,   6178.,   6179.,   6195.,   6199.,   6207.,
         6212.,   6229.,   6290.,   6298.,   6303.,   6307.,   6310.,
         6314.,   6323.,   6325.,   6347.,   6352.,   6354.,   6385.,
         6393.,   6396.,   6399.,   6402.,   6411.,   6432.,   6493.,
         6522.,   6524.,   6536.,   6543.,   6545.,   6554.,   6558.,
         6638.,   66